In [ ]:
import yfinance as yf
import backtrader as bt
import importlib.util
import os
import sys
import traceback

def load_generated_code(file_path):
    try:
        spec = importlib.util.spec_from_file_location("generated_strategy", file_path)
        module = importlib.util.module_from_spec(spec)
        sys.modules["generated_strategy"] = module
        spec.loader.exec_module(module)
        return module
    except Exception as e:
        print(f"Error loading generated code: {e}")
        traceback.print_exc()
        return None

def fetch_sample_data(ticker, start, end):
    try:
        data = yf.download(ticker, start=start, end=end)
        return bt.feeds.PandasData(dataname=data)
    except Exception as e:
        print(f"Error fetching data: {e}")
        traceback.print_exc()
        return None

def run_backtest(strategy_module):
    try:
        # Initialize Cerebro engine
        cerebro = bt.Cerebro()
        # Add strategy
        cerebro.addstrategy(strategy_module.SmaCross)  # Assuming the strategy class is SmaCross
        # Fetch data and add to Cerebro
        data = fetch_sample_data('AAPL', '2020-01-01', '2021-01-01')
        if not data:
            return False
        cerebro.adddata(data)
        # Set initial cash
        cerebro.broker.setcash(10000.0)
        # Run backtest
        cerebro.run()
        # Print results
        print('Ending Portfolio Value: %.2f' % cerebro.broker.getvalue())
        return True
    except Exception as e:
        print(f"Error running backtest: {e}")
        traceback.print_exc()
        return False

def validate_generated_code(file_path):
    # Load the generated code
    strategy_module = load_generated_code(file_path)
    if not strategy_module:
        return False
    # Run the backtest with the loaded strategy
    success = run_backtest(strategy_module)
    return success

if __name__ == "__main__":
    generated_code_path = "path/to/generated_code.py"  # Replace with actual path
    result = validate_generated_code(generated_code_path)
    if result:
        print("Generated code ran successfully.")
    else:
        print("Generated code failed.")
